In [1]:
#!/usr/bin/env python3
import sevenbridges as sbg
from sevenbridges.errors import SbgError
from sevenbridges.http.error_handlers import rate_limit_sleeper, maintenance_sleeper
import sys
import re
import concurrent.futures
import pdb
config = sbg.Config(profile='turbo')
api = sbg.Api(config=config, error_handlers=[rate_limit_sleeper, maintenance_sleeper])

## Run single genotyping wf

### copy refs

In [3]:
src = 'kfdrc-harmonization/sd-bhjxbdqk-09'
dest = 'kfdrc-harmonization/sd-dypmehhf-02'

axiomPoly_resource_vcf = api.files.query(project=src, names=['Axiom_Exome_Plus.genotypes.all_populations.poly.hg38.vcf.gz'])[0]
axiomPoly_resource_vcf.copy(project=dest)
tbi = api.files.query(project=src, names=['Axiom_Exome_Plus.genotypes.all_populations.poly.hg38.vcf.gz.tbi'])[0]
tbi.copy(project=dest)
reference_dict = api.files.query(project=src, names=['Homo_sapiens_assembly38.dict'])[0]
reference_dict.copy(project=dest)
dbsnp_vcf = api.files.query(project=src, names=['Homo_sapiens_assembly38.dbsnp138.vcf'])[0]
dbsnp_vcf.copy(project=dest)
idx = api.files.query(project=src, names=['Homo_sapiens_assembly38.dbsnp138.vcf.idx'])[0]
idx.copy(project=dest)
hapmap_resource_vcf = api.files.query(project=src, names=['hapmap_3.3.hg38.vcf.gz'])[0]
hapmap_resource_vcf.copy(project=dest)
tbi = api.files.query(project=src, names=['hapmap_3.3.hg38.vcf.gz.tbi'])[0]
tbi.copy(project=dest)
mills_resource_vcf = api.files.query(project=src, names=['Mills_and_1000G_gold_standard.indels.hg38.vcf.gz'])[0]
mills_resource_vcf.copy(project=dest)
tbi = api.files.query(project=src, names=['Mills_and_1000G_gold_standard.indels.hg38.vcf.gz.tbi'])[0]
tbi.copy(project=dest)
omni_resource_vcf = api.files.query(project=src, names=['1000G_omni2.5.hg38.vcf.gz'])[0]
omni_resource_vcf.copy(project=dest)
tbi = api.files.query(project=src, names=['1000G_omni2.5.hg38.vcf.gz.tbi'])[0]
tbi.copy(project=dest)
one_thousand_genomes_resource_vcf = api.files.query(project=src, names=['1000G_phase1.snps.high_confidence.hg38.vcf.gz'])[0]
one_thousand_genomes_resource_vcf.copy(project=dest)
tbi = api.files.query(project=src, names=['1000G_phase1.snps.high_confidence.hg38.vcf.gz.tbi'])[0]
tbi.copy(project=dest)
ref_fasta = api.files.query(project=src, names=['Homo_sapiens_assembly38.fasta'])[0]
ref_fasta.copy(project=dest)
fai = api.files.query(project=src, names=['Homo_sapiens_assembly38.fasta.fai'])[0]
fai.copy(project=dest)
snp_sites = api.files.query(project=src, names=['1000G_phase3_v4_20130502.sites.hg38.vcf'])[0]
idx = api.files.query(project=src, names=['1000G_phase3_v4_20130502.sites.hg38.vcf.idx'])[0]
idx.copy(project=dest)
unpadded_intervals_file = api.files.query(project=src, names=['hg38.even.handcurated.20k.intervals'])[0]
unpadded_intervals_file.copy(project=dest)
wgs_evaluation_interval_list = api.files.query(project=src, names=['wgs_evaluation_regions.hg38.interval_list'])[0]
wgs_evaluation_interval_list.copy(project=dest)


<File: id=5ddc4ac9e4b09d9a3261d706>

In [10]:
def get_refs(api):
    ref_dict = {}
    ref_dict['axiomPoly_resource_vcf'] = api.files.query(project=project, names=['Axiom_Exome_Plus.genotypes.all_populations.poly.hg38.vcf.gz'])[0]
    ref_dict['reference_dict'] = api.files.query(project=project, names=['Homo_sapiens_assembly38.dict'])[0]
    ref_dict['dbsnp_vcf'] = api.files.query(project=project, names=['Homo_sapiens_assembly38.dbsnp138.vcf'])[0]
    ref_dict['hapmap_resource_vcf'] = api.files.query(project=project, names=['hapmap_3.3.hg38.vcf.gz'])[0]
    ref_dict['mills_resource_vcf'] = api.files.query(project=project, names=['Mills_and_1000G_gold_standard.indels.hg38.vcf.gz'])[0]
    ref_dict['omni_resource_vcf'] = api.files.query(project=project, names=['1000G_omni2.5.hg38.vcf.gz'])[0]
    ref_dict['one_thousand_genomes_resource_vcf'] = api.files.query(project=project, names=['1000G_phase1.snps.high_confidence.hg38.vcf.gz'])[0]
    ref_dict['reference_fasta'] = api.files.query(project=project, names=['Homo_sapiens_assembly38.fasta'])[0]
    ref_dict['unpadded_intervals_file'] = api.files.query(project=project, names=['hg38.even.handcurated.20k.intervals'])[0]
    ref_dict['wgs_evaluation_interval_list'] = api.files.query(project=project, names=['wgs_evaluation_regions.hg38.interval_list'])[0]
    ref_dict['snp_sites'] = api.files.query(project=project, names=['1000G_phase3_v4_20130502.sites.hg38.vcf'])[0]
    return ref_dict

In [12]:
project = 'kfdrc-harmonization/sd-dypmehhf-02'
manifest = open('/Users/brownm28/Documents/2019-Nov-25_Maris_pedcbio_load/manifests/normal_crams-manifest.csv')
head = next(manifest)
ref_obj = get_refs(api)
app_name = project + "/kfdrc-single-genotype-basic"
for line in manifest:
    info = line.rstrip('\n').split(',')
    pt_id = info[6]
    bs_id = info[11]
    in_dict = {}
    for key in ref_obj:
        in_dict[key] = ref_obj[key]
    related = api.files.query(project='kfdrc-harmonization/sd-dypmehhf-02', metadata={'Kids First Participant ID': pt_id})
    f = 0
    for obj in related:
        if obj.name[-8:] == "g.vcf.gz":
            in_dict['input_vcfs'] = [obj]
            f = 1
            break
    if f == 1:
        task_name = "SINGLE GENOTYPE GATK: " + pt_id + " " + bs_id
        print (task_name)
        task = api.tasks.create(name=task_name, project=project, app=app_name, inputs=in_dict, run=False)
        task.inputs['output_basename'] = task.id
        task.save()
    else:
        sys.stderr.write("GVCF not found for " + pt_id + " skipping!\n")

SINGLE GENOTYPE GATK: PT_S4EJKTME BS_02M67EFQ
SINGLE GENOTYPE GATK: PT_HC1QFR28 BS_05HM0A95
SINGLE GENOTYPE GATK: PT_KRHMQFFP BS_05TA6E00
SINGLE GENOTYPE GATK: PT_00Y8C0XA BS_066WH8B1
SINGLE GENOTYPE GATK: PT_00QYKRAX BS_089VTMQN
SINGLE GENOTYPE GATK: PT_APMAKP20 BS_08WQR40A
SINGLE GENOTYPE GATK: PT_ECTDZ6QS BS_09QP7PQA
SINGLE GENOTYPE GATK: PT_581CW7RN BS_0DT9J904
SINGLE GENOTYPE GATK: PT_ART6DAKP BS_0F489H89
SINGLE GENOTYPE GATK: PT_7APMD0HG BS_0GTTKPSC
SINGLE GENOTYPE GATK: PT_P44YQ044 BS_0J4JR4PC
SINGLE GENOTYPE GATK: PT_E7PFZT6E BS_0JW2Z9AR
SINGLE GENOTYPE GATK: PT_RVTVP55V BS_0PF16SV2
SINGLE GENOTYPE GATK: PT_KWRFGRER BS_0PQA0GGY
SINGLE GENOTYPE GATK: PT_92RR9C8D BS_0QPN2RNE
SINGLE GENOTYPE GATK: PT_Q77S4ZP7 BS_0TVNW0MY
SINGLE GENOTYPE GATK: PT_V1Q9W1NW BS_0V1W9CHW
SINGLE GENOTYPE GATK: PT_TKJ2ZKW6 BS_0XDZKZJ0
SINGLE GENOTYPE GATK: PT_VJ7NKBKF BS_11VVPZTP
SINGLE GENOTYPE GATK: PT_9DD8F0VD BS_14BSNTXX
SINGLE GENOTYPE GATK: PT_VC8M899A BS_17CG1SV7
SINGLE GENOTYPE GATK: PT_V3BXBVVV 

SINGLE GENOTYPE GATK: PT_MG3HP8D9 BS_GRJ3CN0E
SINGLE GENOTYPE GATK: PT_YS31MFW0 BS_GSP3VZ42
SINGLE GENOTYPE GATK: PT_RZPRP3ZC BS_H2VM49X1
SINGLE GENOTYPE GATK: PT_8DFBAQVQ BS_H3TFMS12
SINGLE GENOTYPE GATK: PT_GVWRN4EN BS_H5GG77KB
SINGLE GENOTYPE GATK: PT_NN4RY6A8 BS_H8P4QF0S
SINGLE GENOTYPE GATK: PT_Y18W73AA BS_H8RFBM07
SINGLE GENOTYPE GATK: PT_M4ETZ912 BS_HC01D8JV
SINGLE GENOTYPE GATK: PT_QCMS0C3W BS_HE9SGQRY
SINGLE GENOTYPE GATK: PT_64B8K70Y BS_HG862RR7
SINGLE GENOTYPE GATK: PT_V8AVBQGQ BS_HGNSF36D
SINGLE GENOTYPE GATK: PT_BQCKFXJJ BS_HPFXMMKF
SINGLE GENOTYPE GATK: PT_ZS5D8MVF BS_HQNBAC1K
SINGLE GENOTYPE GATK: PT_B0YZ0H85 BS_HSEZQGA0
SINGLE GENOTYPE GATK: PT_3VNMNFT6 BS_HXFX1HGG
SINGLE GENOTYPE GATK: PT_YBAHTBXV BS_HXKC8MTV
SINGLE GENOTYPE GATK: PT_S700N3ZC BS_J1ETVA6N
SINGLE GENOTYPE GATK: PT_9WX4WBRB BS_J3448NCS
SINGLE GENOTYPE GATK: PT_1WETGRS9 BS_J36GR09E
SINGLE GENOTYPE GATK: PT_WH6RANZQ BS_J77SSWMK
SINGLE GENOTYPE GATK: PT_BQBJFWKZ BS_JBGMTN6S
SINGLE GENOTYPE GATK: PT_RSPKGFXS 

SINGLE GENOTYPE GATK: PT_5W51TAZS BS_ZRYQGHDB
SINGLE GENOTYPE GATK: PT_9R5Y63KX BS_ZT1WRPTB
SINGLE GENOTYPE GATK: PT_09KT7R82 BS_ZXNHCA0E
SINGLE GENOTYPE GATK: PT_R8Q4HK6J BS_ZY0ZDV9Q


## Tag outputs

In [2]:
def mt_tag_outputs(task):
    if re.search(phrase, task.name):
        sys.stderr.write('Valid task found ' + task.name + '\n')
        metadata = task.inputs[in_key].metadata
        for out_key in task.outputs:
            try:
                if type(task.outputs[out_key]) is not list:
                    file_obj = api.files.get(task.outputs[out_key].id)
                    for key in metadata:
                        file_obj.metadata[key] = metadata[key]
                    file_obj.save()
                else:
                    for output in task.outputs[out_key]:
                        file_obj = api.files.get(output.id)
                        for key in metadata:
                            file_obj.metadata[key] = metadata[key]
                        file_obj.save()
            except Exception as e:
                print(e)
                print("Skipping " + task.name + " due to error")

In [3]:
project = 'kfdrc-harmonization/sd-dypmehhf-05'
tasks = api.tasks.query(project=project, status="COMPLETED").all()
# task name search phrase
phrase = "KFDRC CONSENSUS"
# modify this to set which input file to use to tag the outputs with, may need to modify code if an array element
in_key = 'strelka2_vcf'
# for task in tasks:
#     if re.search(phrase, task.name):
#         sys.stderr.write('Valid task found ' + task.name + '\n')
#         metadata = task.inputs[in_key].metadata
#         for out_key in task.outputs:
#             try:
#                 if type(task.outputs[out_key]) is not list:
#                     file_obj = api.files.get(task.outputs[out_key].id)
#                     for key in metadata:
#                         file_obj.metadata[key] = metadata[key]
#                     file_obj.save()
#                 else:
#                     for output in task.outputs[out_key]:
#                         file_obj = api.files.get(output.id)
#                         for key in metadata:
#                             file_obj.metadata[key] = metadata[key]
#                         file_obj.save()
#             except Exception as e:
#                 print(e)
#                 print("Skipping " + task.name + " due to error")
with concurrent.futures.ThreadPoolExecutor(8) as executor:
    results = {executor.submit(mt_tag_outputs, task): task for task in tasks}


Valid task found KFDRC CONSENSUS: PT_VJ7NKBKF BS_GZ39V8AK BS_11VVPZTP
Valid task found KFDRC CONSENSUS: PT_Z4S0193A BS_5ACNB1BG BS_2JKH4PC2
Valid task found KFDRC CONSENSUS: PT_Q77S4ZP7 BS_PDWEETXG BS_0TVNW0MY
Valid task found KFDRC CONSENSUS: PT_3EHTTYB9 BS_QP05TKVG BS_445YTYE8
Valid task found KFDRC CONSENSUS: PT_YK5J3P8F BS_P4C0QJ9K BS_1TWPDAAE
Valid task found KFDRC CONSENSUS: PT_ECTDZ6QS BS_25EXB29C BS_09QP7PQA
Valid task found KFDRC CONSENSUS: PT_S4EJKTME BS_TT4PHFV7 BS_02M67EFQ
Valid task found KFDRC CONSENSUS: PT_VVVS471N BS_SPGHWHXY BS_1S57A8WK
Valid task found KFDRC CONSENSUS: PT_SBS3N6ZT BS_5QFRPR61 BS_2HW3AHX5
Valid task found KFDRC CONSENSUS: PT_00Y8C0XA BS_4ZE1R1FW BS_066WH8B1
Valid task found KFDRC CONSENSUS: PT_2E11C89R BS_9G2RB0V8 BS_2MY5XJ78
Valid task found KFDRC CONSENSUS: PT_KRHMQFFP BS_DZ998H9P BS_05TA6E00
Valid task found KFDRC CONSENSUS: PT_P1MTPZDE BS_SSS5J8P5 BS_55EM1SYG
Valid task found KFDRC CONSENSUS: PT_CZJ3ZTNW BS_7NXB23C9 BS_6FVTM4M0
Valid task found KFD

Valid task found KFDRC CONSENSUS: PT_FKE049G2 BS_DAX3YVG0 BS_E9QPR0QG
Valid task found KFDRC CONSENSUS: PT_J3VD4RFP BS_S8STY9B1 BS_CJ0E9FNC
Valid task found KFDRC CONSENSUS: PT_06Z51EN5 BS_DHC9SHF6 BS_CAXD5ZG6
Valid task found KFDRC CONSENSUS: PT_2RZN4HR2 BS_ZED6T1CT BS_GJTA6W7D
Valid task found KFDRC CONSENSUS: PT_D9XF79J4 BS_30NT6PNA BS_BR40BA47
Valid task found KFDRC CONSENSUS: PT_7E6A5N3P BS_WRZATZQF BS_DK4EQQ3J
Valid task found KFDRC CONSENSUS: PT_53M7K3JE BS_KXRFQF5N BS_C12CV8CF
Valid task found KFDRC CONSENSUS: PT_HZ4VWQP5 BS_SMJRCVQB BS_D0VGGE9Y
Valid task found KFDRC CONSENSUS: PT_3WF5J3PZ BS_ACCE0MEA BS_DBV7S78S
Valid task found KFDRC CONSENSUS: PT_22BQQFYM BS_1G5N5R8S BS_FCP8V8F4
Valid task found KFDRC CONSENSUS: PT_5430SSPY BS_P69A72G2 BS_BVBGZJ1C
Valid task found KFDRC CONSENSUS: PT_ATQMV6B3 BS_E4N511ZX BS_CEXX4K9B
Valid task found KFDRC CONSENSUS: PT_XDPN4357 BS_T9VPRPVA BS_DQ5RVSH4
Valid task found KFDRC CONSENSUS: PT_KBVX8B37 BS_BF83XX7P BS_DB1R553P
Valid task found KFD

Valid task found KFDRC CONSENSUS: PT_GV2XJJTP BS_QDRHQSPZ BS_F0MRP82D
Valid task found KFDRC CONSENSUS: PT_70BK6DFW BS_WCEHT7D0 BS_P9W411VE
Valid task found KFDRC CONSENSUS: PT_E3R0MRXN BS_9ZDR2H7D BS_QA82D9SK
Valid task found KFDRC CONSENSUS: PT_HS4P27SE BS_55PH1SAS BS_QVKVT84J
Valid task found KFDRC CONSENSUS: PT_69AGBVQ5 BS_D7442ACV BS_S2PNHJKQ
Valid task found KFDRC CONSENSUS: PT_K579G3KQ BS_E9YW2KHS BS_RZF7WFQP
Valid task found KFDRC CONSENSUS: PT_D508JGWE BS_P6FPBJM8 BS_PWNFRYW3
Valid task found KFDRC CONSENSUS: PT_FZ3XEWEK BS_VE90VKZY BS_R8696FBC
Valid task found KFDRC CONSENSUS: PT_K36T4AMZ BS_HEP8QTNN BS_QN6EXK06
Valid task found KFDRC CONSENSUS: PT_MK375DCF BS_29MQMSCM BS_SDXP82M8
Valid task found KFDRC CONSENSUS: PT_CCC65GCE BS_8MJNDM75 BS_R1YG2078
Valid task found KFDRC CONSENSUS: PT_05ZMZBF7 BS_31MQXA1J BS_NP9ZEEJX
Valid task found KFDRC CONSENSUS: PT_66Y5KGME BS_9ND96NYZ BS_RXP6GA86
Valid task found KFDRC CONSENSUS: PT_ASJZTDRF BS_NQN1P6QY BS_NVAYK43V
Valid task found KFD

## Run ControlFreeC

In [44]:
def get_freec_refs(api):
    ref_dict = {}
    ref_dict['indexed_reference_fasta'] = api.files.query(project=project, names=['Homo_sapiens_assembly38.fasta'])[0]
    ref_dict['reference_fai'] = api.files.query(project=project, names=['Homo_sapiens_assembly38.fasta.fai'])[0]
    ref_dict['chr_len'] = api.files.query(project=project, names=['hs38_chr.len'])[0]
    ref_dict['ploidy'] = [2,3,4]
    ref_dict['coeff_var'] = 0.05
    ref_dict['threads'] = 16
    return ref_dict

In [45]:
def setup_task(pt_id):
    try:
        in_dict = {}
        for key in ref_objs:
            in_dict[key] = ref_objs[key]
        in_dict['b_allele'] = b_allele_dict[pt_id]
        tum_bs_id = cram_dict[pt_id]['Tumor'].metadata['Kids First Biospecimen ID']
        in_dict['input_tumor_name'] = tum_bs_id
        in_dict['input_tumor'] = cram_dict[pt_id]['Tumor']
        norm_bs_id = cram_dict[pt_id]['Normal'].metadata['Kids First Biospecimen ID']
        in_dict['input_normal'] = cram_dict[pt_id]['Normal']
        pred_sex = sex_dict[pt_id]
        #pdb.set_trace()
        ds_sex = cram_dict[pt_id]['Normal'].metadata['gender']
        if pred_sex != "Unknown" and pred_sex == ds_sex:
            in_dict['cfree_sex'] = s_trans[pred_sex]
        else:
            sys.stderr.write("Warn, prediction for " + case_id + " was inconclusive.  Default to reported sex\n")
            in_dict['cfree_sex'] = s_trans[ds_sex]

        task_name = 'KFDRC CONTROLFREEC: ' + pt_id + " " + tum_bs_id + " " + norm_bs_id
        task = api.tasks.create(name=task_name, project=project, app=app_name, inputs=in_dict, run=False)
        task.inputs['output_basename'] = task.id
        task.save()
    except Exception as e:
        sys.stderr.write(str(e) + " failed processing " + pt_id + "\n")
        sys.exit(1)


In [46]:
project = 'kfdrc-harmonization/sd-dypmehhf-05'
app_name = project + "/kfdrc-controlfreec-wf"
blacklist = open('/Users/brownm28/Documents/2019-Nov-25_Maris_pedcbio_load/BS_ID_blacklist.txt')
sex_pred = open('/Users/brownm28/Documents/2019-Nov-25_Maris_pedcbio_load/KFDRC_sex_prediction_table.txt')
b_allele = open('/Users/brownm28/Documents/2019-Nov-25_Maris_pedcbio_load/manifests/b_allele-manifest.csv')
cram_manifest = open('/Users/brownm28/Documents/2019-Nov-25_Maris_pedcbio_load/manifests/cram-manifest.csv')
head = next(sex_pred)
bs_id_list = []
sex_dict = {}
s_trans = {"Male": "XY", "Female": "XX"}
cram_dict = {}
ref_objs = get_freec_refs(api)
for line in blacklist:
    bs_id_list.append(line.rstrip('\n'))
for line in sex_pred:
    info = line.rstrip('\n').split('\t')
    pt_id = info[2]
    sex = info[1]
    sex_dict[pt_id] = sex
head = next(b_allele)
pt_id_blacklist = []
b_allele_dict = {}
for line in b_allele:
    info = line.rstrip('\n').split(',')
    bs_id = info[11]
    pt_id = info[6]
    if bs_id not in bs_id_list:
        b_allele_dict[pt_id] = api.files.get(info[0])
    else:
        pt_id_blacklist.append(pt_id)
        sys.stderr.write(pt_id + "\tin skip list\n")
sys.stderr.write('Got b allele files\n')
head = next(cram_manifest)
for line in cram_manifest:
    info = line.rstrip('\n').split(',')
    bs_id = info[11]
    pt_id = info[6]
    stype = info[9]
    if pt_id not in pt_id_blacklist:
        if pt_id not in cram_dict:
            cram_dict[pt_id] = {}
        cram_dict[pt_id][stype] = api.files.get(info[0])
sys.stderr.write('Got cram files\n')
i = 1
n = 50
with concurrent.futures.ThreadPoolExecutor(8) as executor:
    results = {executor.submit(setup_task, pt_id): pt_id for pt_id in cram_dict}
    for result in concurrent.futures.as_completed(results):
        if i % n == 0:
            sys.stderr.write(str(i) + ' tasks set up\n')
        i += 1
# for pt_id in cram_dict:
#     setup_task(pt_id)

PT_ECFBP1W1	in skip list
PT_BQBJFWKZ	in skip list
PT_NN4RY6A8	in skip list
PT_R8Q4HK6J	in skip list
PT_VFRRM1BT	in skip list
PT_KSSHMEV6	in skip list
Got b allele files
Got cram files
50 tasks set up
100 tasks set up
150 tasks set up
200 tasks set up
250 tasks set up
300 tasks set up
350 tasks set up


## Consensus calling

In [66]:
def setup_cc_task(pt_id):
    try:
        in_dict = {}
        for key in ref_dict:
            in_dict[key] = ref_dict[key]
        tum_bs_id = vcf_dict[pt_id]['tum_bs_id']
        norm_bs_id = vcf_dict[pt_id]['norm_bs_id']
        in_dict['input_tumor_name'] = tum_bs_id
        in_dict['input_normal_name'] = norm_bs_id
        for tool in tools:
            in_dict[tool + '_vcf'] = vcf_dict[pt_id][tool]
    

        task_name = 'KFDRC CONSENSUS: ' + pt_id + " " + tum_bs_id + " " + norm_bs_id
        task = api.tasks.create(name=task_name, project=project, app=app_name, inputs=in_dict, run=False)
        task.inputs['output_basename'] = task.id
        task.save()
    except Exception as e:
        sys.stderr.write(str(e) + " failed processing " + pt_id + "\n")
        sys.exit(1)


In [67]:
def process_manifest(manifest, tool):
    meta_dict = {'pt_id': 'Kids First Participant ID', 'tum_bs_id': 'Kids First Biospecimen ID Tumor', 'norm_bs_id': 'Kids First Biospecimen ID Normal'}
    meta_keys = {'Kids First Participant ID': 0, 'Kids First Biospecimen ID Normal': 0, 'Kids First Biospecimen ID Tumor': 0}
    f = 0
    head = next(manifest)
    header = head.rstrip('\n').split(',')
    for i in range(0, len(header), 1):
        if header[i] in meta_keys:
            meta_keys[header[i]] = i
            f += 1
    if f != 3:
        sys.stderr.write('Did not find all headers, check manifest!\n')
        sys.exit(1)
    for line in manifest:
        # pdb.set_trace()
        info = line.rstrip('\n').split(',')
        pt_id = info[meta_keys[meta_dict['pt_id']]]
        if pt_id not in blacklist:
            if tool  == 'strelka2':
                vcf_dict[pt_id] = {}
                tum_bs_id = info[meta_keys[meta_dict['tum_bs_id']]]
                norm_bs_id = info[meta_keys[meta_dict['norm_bs_id']]]
                vcf_dict[pt_id]['tum_bs_id'] = tum_bs_id
                vcf_dict[pt_id]['norm_bs_id'] = norm_bs_id
            vcf_dict[pt_id][tool] = api.files.get(info[0])
        else:
            sys.stderr.write(pt_id + " skipped\n")


In [68]:
project = 'kfdrc-harmonization/sd-dypmehhf-05'
app_name = project + "/kfdrc-consensus-calling"
blacklist = open('/Users/brownm28/Documents/2019-Nov-25_Maris_pedcbio_load/pt_id_blacklist.txt')
strelka2_manifest = open('/Users/brownm28/Documents/2019-Nov-25_Maris_pedcbio_load/manifests/strelka2-manifest.csv')
mutect2_manifest = open('/Users/brownm28/Documents/2019-Nov-25_Maris_pedcbio_load/manifests/mutect2-manifest.csv')
lancet_manifest = open('/Users/brownm28/Documents/2019-Nov-25_Maris_pedcbio_load/manifests/lancet-manifest.csv')
vardict_manifest = open('/Users/brownm28/Documents/2019-Nov-25_Maris_pedcbio_load/manifests/vardict-manifest.csv')
ref_dict = {}
ref_dict['indexed_reference_fasta'] = api.files.query(project=project, names=['Homo_sapiens_assembly38.fasta'])[0]
ref_dict['vep_cache'] = api.files.query(project=project, names=['homo_sapiens_vep_93_GRCh38_convert_cache.tar.gz'])[0]
ref_dict['strip_info'] = "INFO/CSQ"
pt_skip = []
for pt_id in blacklist:
    pt_skip.append(pt_id.rstrip('\n'))
vcf_dict = {}
# process strelka2 first so that dict is properly initialized
process_manifest(strelka2_manifest, 'strelka2')
process_manifest(mutect2_manifest, 'mutect2')
process_manifest(lancet_manifest, 'lancet')
process_manifest(vardict_manifest, 'vardict')

i = 1
n = 50
tools = ['strelka2', 'mutect2', 'lancet', 'vardict']
with concurrent.futures.ThreadPoolExecutor(8) as executor:
    results = {executor.submit(setup_cc_task, pt_id): pt_id for pt_id in vcf_dict}
    for result in concurrent.futures.as_completed(results):
        if i % n == 0:
            sys.stderr.write(str(i) + ' tasks set up\n')
        i += 1



50 tasks set up
100 tasks set up
150 tasks set up
200 tasks set up
250 tasks set up
300 tasks set up
350 tasks set up


## Tag paired outputs with task inputs

In [70]:
project = 'kfdrc-harmonization/sd-dypmehhf-05'
tasks = api.tasks.query(project=project, status="COMPLETED").all()
# task name search phrase
phrase = "KFDRC CONTROLFREEC"
# modify this to set which input file to use to tag the outputs with, may need to modify code if an array element
tumor_in_key = 'input_tumor_aligned'
normal_in_key = 'input_normal_aligned'
for task in tasks:
    if re.search(phrase, task.name):
        sys.stderr.write('Valid task found ' + task.name + '\n')
        metadata = task.inputs[tumor_in_key].metadata
        # del metadata['Kids First Biospecimen ID']
        metadata['Kids First Biospecimen ID Normal'] = task.inputs[normal_in_key].metadata['Kids First Biospecimen ID']
        metadata['Kids First Biospecimen ID Tumor'] = task.inputs[tumor_in_key].metadata['Kids First Biospecimen ID']
        for out_key in task.outputs:
            try:
                if type(task.outputs[out_key]) is not list:
                    file_obj = api.files.get(task.outputs[out_key].id)
                    for key in metadata:
                        file_obj.metadata[key] = metadata[key]
                    file_obj.save()
                else:
                    for output in task.outputs[out_key]:
                        file_obj = api.files.get(output.id)
                        for key in metadata:
                            file_obj.metadata[key] = metadata[key]
                        file_obj.save()
            except Exception as e:
                print(e)
                print("Skipping " + task.name + " due to error")

Valid task found KFDRC CONTROLFREEC: PT_S4EJKTME BS_TT4PHFV7 BS_02M67EFQ
Valid task found KFDRC CONTROLFREEC: PT_ECTDZ6QS BS_25EXB29C BS_09QP7PQA
Valid task found KFDRC CONTROLFREEC: PT_581CW7RN BS_J6FPW4H0 BS_0DT9J904
Valid task found KFDRC CONTROLFREEC: PT_HC1QFR28 BS_XQC9PNZP BS_05HM0A95
Valid task found KFDRC CONTROLFREEC: PT_00Y8C0XA BS_4ZE1R1FW BS_066WH8B1
Valid task found KFDRC CONTROLFREEC: PT_00QYKRAX BS_CG9KDK95 BS_089VTMQN
Valid task found KFDRC CONTROLFREEC: PT_APMAKP20 BS_CKVK1AYV BS_08WQR40A
Valid task found KFDRC CONTROLFREEC: PT_ART6DAKP BS_X242DJMF BS_0F489H89
Valid task found KFDRC CONTROLFREEC: PT_7APMD0HG BS_T5BAS1JN BS_0GTTKPSC
Valid task found KFDRC CONTROLFREEC: PT_P44YQ044 BS_8C5QG3GE BS_0J4JR4PC
Valid task found KFDRC CONTROLFREEC: PT_E7PFZT6E BS_NK1FJNRG BS_0JW2Z9AR
Valid task found KFDRC CONTROLFREEC: PT_RVTVP55V BS_JV1GTZ1Q BS_0PF16SV2
Valid task found KFDRC CONTROLFREEC: PT_KWRFGRER BS_4RX1AAVV BS_0PQA0GGY
Valid task found KFDRC CONTROLFREEC: PT_92RR9C8D BS

Valid task found KFDRC CONTROLFREEC: PT_ZK8Z4WAK BS_XA3KEKC6 BS_AWW96J4B
Valid task found KFDRC CONTROLFREEC: PT_8TSFEMRP BS_BHKAK446 BS_AKFZ2WVR
Valid task found KFDRC CONTROLFREEC: PT_AE730SVH BS_D8H43Q28 BS_B7NS4GEE
Valid task found KFDRC CONTROLFREEC: PT_PV869ZYE BS_TWN989ZD BS_BFSZW2CB
Valid task found KFDRC CONTROLFREEC: PT_8HFHWZH9 BS_KE5D12KG BS_BBMTN7JM
Valid task found KFDRC CONTROLFREEC: PT_GA9F5STK BS_QSTNH2MJ BS_BQ7KF7H1
Valid task found KFDRC CONTROLFREEC: PT_D9XF79J4 BS_30NT6PNA BS_BR40BA47
Valid task found KFDRC CONTROLFREEC: PT_AE9Q9RMT BS_CP8741SX BS_BNPXDDS7
Valid task found KFDRC CONTROLFREEC: PT_0SMDNDVT BS_VVWYPEHM BS_BRQMRW46
Valid task found KFDRC CONTROLFREEC: PT_W6JHGQWM BS_PRMXCJDJ BS_BSRMGGMV
Valid task found KFDRC CONTROLFREEC: PT_5430SSPY BS_P69A72G2 BS_BVBGZJ1C
Valid task found KFDRC CONTROLFREEC: PT_QW5Q0G84 BS_HCWEF4QN BS_BZW66BK2
Valid task found KFDRC CONTROLFREEC: PT_53M7K3JE BS_KXRFQF5N BS_C12CV8CF
Valid task found KFDRC CONTROLFREEC: PT_FW0K9SXK BS

Valid task found KFDRC CONTROLFREEC: PT_RG7MMHFF BS_W41Y820G BS_NHJ3QDZH
Valid task found KFDRC CONTROLFREEC: PT_AQS8CCAB BS_MW0JYGG1 BS_NHVKRX7J
Valid task found KFDRC CONTROLFREEC: PT_4WVGKQRX BS_7W42FH5A BS_NMN0R7R2
Valid task found KFDRC CONTROLFREEC: PT_3TE71Z5R BS_1BR81RGD BS_NK1JG9GP
Valid task found KFDRC CONTROLFREEC: PT_CZ592107 BS_H18ZNTNR BS_NNKD5B9Y
Valid task found KFDRC CONTROLFREEC: PT_05ZMZBF7 BS_31MQXA1J BS_NP9ZEEJX
Valid task found KFDRC CONTROLFREEC: PT_YPK89ADE BS_SSHQFYA8 BS_NTCN3WCM
Valid task found KFDRC CONTROLFREEC: PT_ASJZTDRF BS_NQN1P6QY BS_NVAYK43V
Valid task found KFDRC CONTROLFREEC: PT_H3QX7Y9N BS_9YMT5HB4 BS_NZCJMYH9
Valid task found KFDRC CONTROLFREEC: PT_1YAJEAMJ BS_RA5HNMDP BS_NVCBHA84
Valid task found KFDRC CONTROLFREEC: PT_EXZSSRGH BS_NVJJJ7MT BS_NWTKPAV9
Valid task found KFDRC CONTROLFREEC: PT_77DSBXEM BS_MYVQ8YDT BS_P2T68QZV
Valid task found KFDRC CONTROLFREEC: PT_2HCWZNTR BS_VKK21QBZ BS_P93093QW
Valid task found KFDRC CONTROLFREEC: PT_SPZNB4QW BS

Valid task found KFDRC CONTROLFREEC: PT_WG51EA8V BS_A3K94FCY BS_YVYDDWG6
Valid task found KFDRC CONTROLFREEC: PT_D1837NS6 BS_QEKBZRMD BS_YX6ZVEXM
Valid task found KFDRC CONTROLFREEC: PT_NK39AFPG BS_R13WNSAR BS_YXZ0BVJ0
Valid task found KFDRC CONTROLFREEC: PT_R07QYFJ0 BS_EDGSP4V6 BS_YZQMGVJ6
Valid task found KFDRC CONTROLFREEC: PT_SZN83NXT BS_B4GBBM0J BS_Z344VSXG
Valid task found KFDRC CONTROLFREEC: PT_W50HSNE2 BS_GPNWSW71 BS_Z40644EA
Valid task found KFDRC CONTROLFREEC: PT_39TTDEKT BS_3CVTNBQ3 BS_Z571SQ06
Valid task found KFDRC CONTROLFREEC: PT_SRAC0TXH BS_S0T1WT6R BS_ZB0BQ3P1
Valid task found KFDRC CONTROLFREEC: PT_5BJXQ731 BS_01BDZ0TW BS_ZBJCX3BC
Valid task found KFDRC CONTROLFREEC: PT_4W8PD8TR BS_GAA781GQ BS_ZGC57N70
Valid task found KFDRC CONTROLFREEC: PT_HB9JT4G5 BS_95PV1RRY BS_ZH140J6Z
Valid task found KFDRC CONTROLFREEC: PT_5W51TAZS BS_J3707KE2 BS_ZRYQGHDB
Valid task found KFDRC CONTROLFREEC: PT_9R5Y63KX BS_56CA6BFQ BS_ZT1WRPTB
Valid task found KFDRC CONTROLFREEC: PT_09KT7R82 BS

### expand view

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))

### extract ploidy info

In [8]:
project = "kfdrc-harmonization/sd-dypmehhf-05"
manifest = open('/Users/brownm28/Documents/2019-Nov-25_Maris_pedcbio_load/manifests/cfree_info-manifest.csv')
skip = next(manifest)
out = open('/Users/brownm28/Documents/2019-Nov-25_Maris_pedcbio_load/maris_cfree_info.txt', 'w')
out.write('Case ID\tWindow Size\tPloidy\tGood Fit\n')
for line in manifest:
    info = line.rstrip('\n').split(',')
    case_id = info[-2]
    file_obj = api.files.get(info[0])
    file_contents = file_obj.content().split('\n')
    ws = file_contents[7].split('\t')
    p = file_contents[-4].split('\t')
    fit = file_contents[-2].split('\t')
    # pdb.set_trace()
    out.write(case_id + "\t" + ws[1] + "\t" + p[1] + "\t" + fit[1] + "\n")
out.close()

# Sample match troubleshoot

## Collate and add ext IDs and FM ids to peddy results

In [7]:
fdir = "/Users/brownm28/Documents/2018-Sep-25_Maris_FY16/peddy_results/"
manifest = "peddy_check_csv-manifest.csv"
# easiest to use BS ID list, and run sample info function from other workbook
ext_id_table = "ext_id_info.txt"
ext_man = open(fdir + ext_id_table)
bs_dict = {}
skip = next(ext_man)
for line in ext_man:
    info = line.rstrip('\n').split("\t")
    bs_dict[info[0]] = info[-1]
ext_man.close()
cav_man = open(fdir + manifest)
head = next(cav_man)
out = open(fdir + "collated_results.txt", "w")
out.write("kf_fam_id\text_fam_id\tkf_sample_a_id\text_sample_a_id\tkf_sample_b_id\text_sample_b_id\tn\tshared_hets\thets_a\tibs2\trel\thets_b\tibs0\tpedigree_parents\tpedigree_relatedness\tpredicted_parents\tparent_error\tsample_duplication_error\trel_difference\n")
for line in cav_man:
    (fid, fname, project, kf_fm, ext_fm) = line.rstrip('\n').split(',')
    fh = open(fdir + "ped_check_csv/" + fname)
    head = next(fh)
    for i in range(0,3,1):
        data = next(fh)
        datum = data.split(",")
        out.write("\t".join([kf_fm, ext_fm, datum[0], bs_dict[datum[0]], datum[1], bs_dict[datum[1]]]) + "\t" + "\t".join(datum[2:]))
    fh.close()
out.close()
        
        

### get task names

In [10]:
tasks = api.tasks.query(project="kfdrc-harmonization/sd-dypmehhf-01", status="COMPLETED").all()
for task in tasks:
    if re.search("Topoff", task.name):
        print (task.id + "\t" + task.name)

b83f2a27-1aeb-4708-8e63-84cfcd033e83	jointgenotyping_and_refinement-Topoff-FM_6R8X2WRY
d69c2a59-90f6-4e37-9627-20449072ecd0	jointgenotyping_and_refinement-Topoff-FM_88AJZKFG
bc70b8fd-a008-43bc-b724-91cfa58f2cd6	jointgenotyping_and_refinement-Topoff-FM_D259QFHW
81dda714-175e-4e51-8ca7-39373e02e5d9	jointgenotyping_and_refinement-Topoff-FM_JV31DYR5
dbf695a3-5e85-404d-9ff9-371f39738653	jointgenotyping_and_refinement-Topoff-FM_MQ90643N
d5bf8548-6283-44d9-b71b-ea766aae2f16	jointgenotyping_and_refinement-Topoff-FM_04BZFE4G
e26257c9-8472-4fd5-9fb9-042e4f023113	jointgenotyping_and_refinement-Topoff-FM_068BMXVN
0c8d884d-4628-4c91-861b-14be0b90cf04	jointgenotyping_and_refinement-Topoff-FM_0F9JFCQJ
040939d3-62ca-4c58-82da-645ddbe24dcb	jointgenotyping_and_refinement-Topoff-FM_0QZHJ596
ed774942-3a0a-4c2f-a07d-0f5fd23c461e	jointgenotyping_and_refinement-Topoff-FM_0S71PRSG
7e5e41e7-f222-41d9-a5a3-602c6f7b2828	jointgenotyping_and_refinement-Topoff-FM_10V2K6AD
cf340497-44a5-49c4-9396-ffc103850a86	jointg

aedd6c3a-ee39-41c5-b956-1132fe2c8fb6	jointgenotyping_and_refinement-Topoff-FM_P23WGGYA
fb52a9f2-dde1-4955-8e2f-80729164a781	jointgenotyping_and_refinement-Topoff-FM_P2P0N9GY
91b8b41e-d2fb-4f74-8a25-5b30b39dc15b	jointgenotyping_and_refinement-Topoff-FM_P5JX9P4J
15a25b24-2c5f-415a-9c98-75a79f719728	jointgenotyping_and_refinement-Topoff-FM_P66RTRC6
6dc2d92e-0d13-4ab3-b132-cb405aac7f17	jointgenotyping_and_refinement-Topoff-FM_PTEDE0E5
bf10e1a1-323b-4f5c-a2e8-714b2a46908b	jointgenotyping_and_refinement-Topoff-FM_QE90NKS3
cdb2ac5b-be22-43ca-a7c5-d52038f0b00b	jointgenotyping_and_refinement-Topoff-FM_QF581032
1f6e24a7-17f3-4153-a40e-5471c6b15be1	jointgenotyping_and_refinement-Topoff-FM_QHXM0Q40
f3afe8d2-6089-43c9-93f5-d15d1a7a0c67	jointgenotyping_and_refinement-Topoff-FM_QNFTR3R4
ff9f2724-8d49-486c-b1c4-688a176b6c62	jointgenotyping_and_refinement-Topoff-FM_QQB5AXNH
aab7ac98-5eb6-4744-a14b-ce2afc68a443	jointgenotyping_and_refinement-Topoff-FM_R24BWA7N
cca0c0cb-8b6b-4543-88a8-61756824ce67	jointg

# Investigate ngs check pass with peddy fail

## Get existing bcf checkmate files

In [2]:
case_ids = open('/Users/brownm28/Documents/2018-Sep-25_Maris_FY16/ngs_checkmate/ngs_pass_peddy_Fail_check/case_id_list.txt')
manifest = open('/Users/brownm28/Documents/2018-Sep-25_Maris_FY16/ngs_checkmate/ngs_pass_peddy_Fail_check/existing_bcf_calls-manifest.csv')
case_list = []
for line in case_ids:
    case_list.append(line.rstrip('\n'))
case_ids.close()
out = open('/Users/brownm28/Documents/2018-Sep-25_Maris_FY16/ngs_checkmate/ngs_pass_peddy_Fail_check/bcf_to_cp.csv', 'w')

head = next(manifest)
out.write(head)
header = head.rstrip('\n').split(',')
c_idx = 0
for i in range(0, len(header), 1):
    if header[i] == 'case_id':
        c_idx = i
        break
for line in manifest:
    info = line.rstrip('\n').split(',')
    if info[c_idx] in case_list:
        out.write(line)
out.close()

## Get crams to test

In [4]:
case_ids = open('/Users/brownm28/Documents/2018-Sep-25_Maris_FY16/ngs_checkmate/ngs_pass_peddy_Fail_check/case_id_list.txt')
manifest = open('/Users/brownm28/Documents/2018-Sep-25_Maris_FY16/ngs_checkmate/ngs_pass_peddy_Fail_check/germline_cram-manifest.csv')
case_list = []
for line in case_ids:
    case_id = line.rstrip('\n')
    case_root = case_id[0:6]
    case_list.append(case_root)
case_ids.close()
out = open('/Users/brownm28/Documents/2018-Sep-25_Maris_FY16/ngs_checkmate/ngs_pass_peddy_Fail_check/cram_to_cp.csv', 'w')

head = next(manifest)
out.write(head)
header = head.rstrip('\n').split(',')
c_idx = 0
for i in range(0, len(header), 1):
    if header[i] == 'case_id':
        c_idx = i
        break
for line in manifest:
    info = line.rstrip('\n').split(',')
    case_root = info[c_idx][0:6]
    if case_root in case_list and info[c_idx][-2:] != "03":
        out.write(line)
out.close()

## Run bcf

In [8]:
def get_ref_objs(api):
    refs = {}
    refs['chr_list'] = api.files.get(id='5c6364d7e4b07ea2b9d53cd6')
    refs['reference_fasta'] = api.files.get(id='5c5debb8e4b07ea205da9cdb')
    refs['snp_bed'] = api.files.get(id='5c5deb36e4b07ea205da9cd6')
    return refs

In [10]:
project="brownm28/ngs-checkmate-dev"
manifest = open('/Users/brownm28/Documents/2018-Sep-25_Maris_FY16/ngs_checkmate/ngs_pass_peddy_Fail_check/cram_to_run-manifest.csv')
head = next(manifest)
cram_list = []

for line in manifest:
    info = line.rstrip('\n').split(',')
    cram_list.append(info[0])
cram_obj_list = []
files = api.files.query(project=project).all()
for fobj in files:
    if fobj.id in cram_list:
        cram_obj_list.append(fobj)
#pdb.set_trace()
app_name = project + '/bcf-call'
ref_obj = get_ref_objs(api)
total = len(cram_obj_list)
ct = 1
#bcf_tasks = open('/Users/brownm28/Documents/2019-Aug-22_cbttc_chop_hakon_checkmate/hakon_bcf_tasks.txt', 'w')
max_j = 6
for i in range(0, total, max_j):
    # pdb.set_trace()
    inputs = ref_obj
    cset = i + max_j
    if cset > total:
        cset = total
    inputs['input_align'] = cram_obj_list[i:cset]
    task_name = 'MARIS PARENT BCF CALLS: SET' + str(ct)
    task = api.tasks.create(name=task_name, project=project, app=app_name, inputs=inputs, run=False)
    task.save()
    # bcf_tasks.write(task.name + '\t' + task.id + '\n')
    ct += 1
# bcf_tasks.close()   

### add metadata to bcf outputs

In [2]:
project="brownm28/ngs-checkmate-dev"
tasks = api.tasks.query(project=project, status="COMPLETED").all()
# task name search phrase
phrase = "MARIS PARENT BCF CALLS"
# modify this to set which input file to use to tag the outputs with, may need to modify code if an array element
in_key = 'input_align'
for task in tasks:
    if re.search(phrase, task.name):
        sys.stderr.write('Valid task found ' + task.name + '\n')
        fset = task.inputs[in_key]
        for i in range(len(fset)):
            metadata = fset[i].metadata
            for out_key in task.outputs:
                try:
                    file_obj = api.files.get(task.outputs[out_key][i])
                    for key in metadata:
                        file_obj.metadata[key] = metadata[key]
                    file_obj.save()
                except Exception as e:
                    print(e)
                    print("Skipping " + task.name + " due to error")

Valid task found MARIS PARENT BCF CALLS: SET1
Valid task found MARIS PARENT BCF CALLS: SET2
Valid task found MARIS PARENT BCF CALLS: SET3
Valid task found MARIS PARENT BCF CALLS: SET4
Valid task found MARIS PARENT BCF CALLS: SET5


### add tags to bcf outputs

In [3]:
project="brownm28/ngs-checkmate-dev"
tags = ['MARIS', 'INVESTIGATE', 'PARENT']
tasks = api.tasks.query(project=project, status="COMPLETED").all()
# task name search phrase
phrase = "MARIS PARENT BCF CALLS"
# modify this to set which input file to use to tag the outputs with, may need to modify code if an array element
in_key = 'input_align'
for task in tasks:
    if re.search(phrase, task.name):
        sys.stderr.write('Valid task found ' + task.name + '\n')
        fset = task.inputs[in_key]
        for i in range(len(fset)):
            for out_key in task.outputs:
                try:
                    file_obj = api.files.get(task.outputs[out_key][i])
                    file_obj.tags = tags
                    file_obj.save()
                except Exception as e:
                    print(e)
                    print("Skipping " + task.name + " due to error")

Valid task found MARIS PARENT BCF CALLS: SET1
Valid task found MARIS PARENT BCF CALLS: SET2
Valid task found MARIS PARENT BCF CALLS: SET3
Valid task found MARIS PARENT BCF CALLS: SET4
Valid task found MARIS PARENT BCF CALLS: SET5


## Run ngs checkmate

In [6]:
def populate_vcf(line):
    info = line.rstrip('\n').split(',')
    sample_id = info[8]
    # GMKF-30-PATTFE06-12A-01D
    case_root = sample_id[8:14]
    if case_root not in vcfs:
        vcfs[case_root] = []
    vcfs[case_root].append(api.files.get(info[0]))

In [7]:
manifest = open('/Users/brownm28/Documents/2018-Sep-25_Maris_FY16/ngs_checkmate/ngs_pass_peddy_Fail_check/maris_vcf_fam-manifest.csv')
head = next(manifest)
vcfs = {}
with concurrent.futures.ThreadPoolExecutor(8) as executor:
    results = {executor.submit(populate_vcf, entry): entry for entry in manifest}

inputs = {}
inputs['snp_bed'] = api.files.get(id='5c5deb36e4b07ea205da9cd6')
inputs['input_vcf'] = []
inputs['output_basename'] = []
for case_root in vcfs:
    if len(vcfs[case_root]) <= 1:
        sys.stderr.write('Only one sample for ' + case_root + " skipping!\n")
    else:
        inputs['input_vcf'].append(vcfs[case_root])
        inputs['output_basename'].append(case_root + "_pool")
task_name = 'MARIS FAM CHECKMATE PASS PEDDY FAIL'
app_name = project + '/ngs-checkmate-wf'
task = api.tasks.create(name=task_name, project=project, app=app_name, inputs=inputs, run=False)
task.save()

<Task: id=44323fda-2240-46ae-8c2a-5f12a487dbc0>

### Add metadata to calls

In [10]:
manifest = open('/Users/brownm28/Documents/2018-Sep-25_Maris_FY16/ngs_checkmate/ngs_pass_peddy_Fail_check/maris_vcf_fam-manifest.csv')
head = next(manifest)
file_meta_dict = {}
for line in manifest:
    info = line.rstrip('\n').split(',')
    fname = info[1]
    pt_id = info[6]
    sample_id = info[8]
    bs_id = info[11]
    fm_id = info[12]
    sample_type = info[9]
    file_meta_dict[fname] = "\t".join([sample_id, fm_id, pt_id, bs_id, sample_type])
manifest.close()

out = open('/Users/brownm28/Documents/2018-Sep-25_Maris_FY16/ngs_checkmate/ngs_pass_peddy_Fail_check/Maris_ngs_pass_peddy_fail_pooled.txt', 'w')
out.write("Status\tCorrelation\tSample1 ID\tS1 KF Family ID\tS1 KF Participant Id\tS1 KF Biospecimen ID\tS1 Sample Type\tSample2 ID\tS2 KF Family ID\tS2 KF Participant Id\tS2 KF Biospecimen ID\tS2 Sample Type\n")
for line in open('/Users/brownm28/Documents/2018-Sep-25_Maris_FY16/ngs_checkmate/ngs_pass_peddy_Fail_check/results/results_contcat.txt'):
    info = line.rstrip('\n').split('\t')
    out.write("\t".join([info[1], info[-2], file_meta_dict[info[0]], file_meta_dict[info[2]]]) + "\n")
out.close()
